In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [95]:
dataframe = pandas.read_csv("train_without_smote_mj_allcat.csv", header=0)
#dataframe_shuffled = dataframe.sample(frac=1).reset_index(drop=True)
#dataset = dataframe.values
#dataset = dataframe_shuffled.values
dataset = dataframe.values


# split into input (X) and output (Y) variables
X = dataset[:,2:].astype(float)
Y = dataset[:,1]
id = dataset[:,0]

#For softmax function
Y =np_utils.to_categorical(Y)

In [96]:
#Number of ones = 21,694
one_count =0
for i in Y:
    if i[1]==1:
        one_count +=1
print one_count        

21694


In [97]:
scale = StandardScaler()
X = scale.fit_transform(X)
print X

[[ 0.0502179   0.95829073  0.21359383 ...,  1.65447572 -0.99693018
  -1.1154726 ]
 [-0.45386836 -0.54166362  0.95436209 ..., -0.60442108  1.00307928
   0.89648101]
 [ 1.56247669  3.95819943  1.69513035 ...,  1.65447572  1.00307928
  -1.1154726 ]
 ..., 
 [-0.45386836 -0.54166362  2.06551448 ..., -0.60442108 -0.99693018
   0.89648101]
 [ 1.56247669  0.95829073 -0.52717443 ..., -0.60442108  1.00307928
   0.89648101]
 [-0.95795462 -0.54166362  1.32474622 ..., -0.60442108 -0.99693018
  -1.1154726 ]]


In [98]:
model = Sequential()
model.add(Dense(85, input_dim=85, kernel_initializer='normal', activation='relu'))
model.add(Dense(40, kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='relu'))
#model.add(Dense(5, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
#Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=500, batch_size =2000)

Epoch 1/500
595212/595212 [==============================] - 3s - loss: 0.2181 - acc: 0.9608     
Epoch 2/500
595212/595212 [==============================] - 3s - loss: 0.1524 - acc: 0.9636     
Epoch 3/500
595212/595212 [==============================] - 3s - loss: 0.1520 - acc: 0.9636     
Epoch 4/500
595212/595212 [==============================] - 3s - loss: 0.1518 - acc: 0.9636     
Epoch 5/500
595212/595212 [==============================] - 3s - loss: 0.1515 - acc: 0.9636     
Epoch 6/500
595212/595212 [==============================] - 3s - loss: 0.1514 - acc: 0.9636     
Epoch 7/500
595212/595212 [==============================] - 3s - loss: 0.1513 - acc: 0.9636     
Epoch 8/500
595212/595212 [==============================] - 3s - loss: 0.1512 - acc: 0.9636     
Epoch 9/500
595212/595212 [==============================] - 3s - loss: 0.1511 - acc: 0.9636     
Epoch 10/500
595212/595212 [==============================] - 3s - loss: 0.1511 - acc: 0.9636     
Epoch 11/500
595212

595212/595212 [==============================] - 3s - loss: 0.1434 - acc: 0.9637     
Epoch 84/500
595212/595212 [==============================] - 3s - loss: 0.1433 - acc: 0.9637     
Epoch 85/500
595212/595212 [==============================] - 3s - loss: 0.1432 - acc: 0.9637     
Epoch 86/500
595212/595212 [==============================] - 3s - loss: 0.1431 - acc: 0.9637     
Epoch 87/500
595212/595212 [==============================] - 3s - loss: 0.1431 - acc: 0.9637     
Epoch 88/500
595212/595212 [==============================] - 3s - loss: 0.1430 - acc: 0.9637     
Epoch 89/500
595212/595212 [==============================] - 3s - loss: 0.1428 - acc: 0.9636     
Epoch 90/500
595212/595212 [==============================] - 3s - loss: 0.1429 - acc: 0.9637     
Epoch 91/500
595212/595212 [==============================] - 3s - loss: 0.1428 - acc: 0.9637     
Epoch 92/500
595212/595212 [==============================] - 3s - loss: 0.1428 - acc: 0.9637     
Epoch 93/500
595212/595

KeyboardInterrupt: 

In [99]:
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

594592/595212 [============================>.] - ETA: 0s
acc: 96.38%


In [100]:
answers_prob = model.predict_proba(X)

594432/595212 [============================>.] - ETA: 0s

In [101]:
predicted_one_count = 0
for i in answers_prob:
    if i[1] >=0.5:
        predicted_one_count +=1 

print predicted_one_count        

532


In [102]:
#Number of ones
print float(predicted_one_count)/float(one_count) * 100

2.45229095602


In [103]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 85)                7310      
_________________________________________________________________
dense_51 (Dense)             (None, 40)                3440      
_________________________________________________________________
dense_52 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_53 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_54 (Dense)             (None, 2)                 22        
Total params: 11,802
Trainable params: 11,802
Non-trainable params: 0
_________________________________________________________________


In [104]:
#Read and prdict the test data
dataframe = pandas.read_csv("test_without_smote_mj_allcat.csv", header=0)
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,1:].astype(float)
#Y = dataset[:,22]
id = dataset[:,0].astype(int)

In [105]:
X = scale.fit_transform(X)
print X
print id.shape

[[-0.9576486  -0.54237645  1.32817405 ..., -0.6037182  -0.99907526
   0.89549453]
 [ 1.05594129  0.96140246  0.2171238  ..., -0.6037182   1.00092559
   0.89549453]
 [ 1.55933876 -0.54237645 -0.52357636 ...,  1.65640193  1.00092559
  -1.1167014 ]
 ..., 
 [-0.9576486  -0.54237645  0.2171238  ..., -0.6037182   1.00092559
   0.89549453]
 [ 2.06273623 -0.54237645  0.2171238  ..., -0.6037182   1.00092559
  -1.1167014 ]
 [ 2.5661337  -0.54237645 -0.15322628 ...,  1.65640193 -0.99907526
  -1.1167014 ]]
(892816,)


In [106]:
answers_prob_test = model.predict_proba(X)

890624/892816 [============================>.] - ETA: 0s

In [107]:
one_probabs_test = answers_prob_test[:,1]

In [108]:
one_probabs_test.shape

(892816,)

In [109]:
sum(one_probabs_test>=0.5)

842

In [110]:
df = pandas.DataFrame({"id":id})
df
df["target"] = one_probabs_test
df.to_csv("answers_full_nn.csv",index=False)